In [1]:
#библиотека для запроса к странице (и получения её кода)
import requests

In [2]:
#поключаем библиотеку fake-useragent, чтобы сервер думал, что мы смотрим сайт не с питона, а с обычного браузера
from fake_useragent import UserAgent
ua = UserAgent()
ua.chrome

'Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36'

In [66]:
#вторая попытка подключиться к серверу
last_page='https://www.domofond.ru/prodazha-kvartiry-moskva-c3584?Rooms=One%2CTwo%2CThree%2CFour%2CFive%2CSix%2CSeven%2CEight%2CNine%2CMoreThan9%2CStudio&Page=2353'
response=requests.get(last_page,headers={'User-Agent': UserAgent().chrome})
response

<Response [200]>

In [67]:
#смотрим на полученный код страницы
html=response.content
html[:1000]

b'\n            <!DOCTYPE html>\n            <html>\n            <head>\n                <meta charset="utf-8">\n                <link rel="shortcut icon" type="image/x-icon" href="/favicon.ico"/>\n                <link rel="apple-touch-icon" href="/static/images/apple/apple-touch-icon-60x60.png">\n                <link rel="apple-touch-icon" sizes="76x76" href="/static/images/apple/apple-touch-icon-76x76.png">\n                <link rel="apple-touch-icon" sizes="120x120" href="/static/images/apple/apple-touch-icon-120x120.png">\n                <link rel="apple-touch-icon" sizes="152x152" href="/static/images/apple/apple-touch-icon-152x152.png">\n                \n            <!-- Google Tag Manager -->\n            <noscript><iframe src="//www.googletagmanager.com/ns.html?id=GTM-MZPV7N"\n            height="0" width="0" style="display:none;visibility:hidden"></iframe></noscript>\n            <script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({\'gtm.start\':\n            new Date().

In [68]:
#библиотека для парсинга (краулинг + скрапинг)
from bs4 import BeautifulSoup

In [69]:
#преобразуем код страницы через BeautifulSoup
soup = BeautifulSoup(html,"html.parser")

In [70]:
#первые 1000 символов кода страницы
print(str(soup)[:1000])


<!DOCTYPE html>

<html>
<head>
<meta charset="utf-8"/>
<link href="/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/static/images/apple/apple-touch-icon-60x60.png" rel="apple-touch-icon"/>
<link href="/static/images/apple/apple-touch-icon-76x76.png" rel="apple-touch-icon" sizes="76x76"/>
<link href="/static/images/apple/apple-touch-icon-120x120.png" rel="apple-touch-icon" sizes="120x120"/>
<link href="/static/images/apple/apple-touch-icon-152x152.png" rel="apple-touch-icon" sizes="152x152"/>
<!-- Google Tag Manager -->
<noscript><iframe height="0" src="//www.googletagmanager.com/ns.html?id=GTM-MZPV7N" style="display:none;visibility:hidden" width="0"></iframe></noscript>
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
            new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
            j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
            '//www.googletagmanager.com/gtm.js?id='+i+dl;f.parent

In [78]:
j_number=0
for j in range(len(soup.findAll("script"))):
    if str(soup.findAll("script")[j]).find("window.__INITIAL_DATA__ =")>0:
        j_number=j
        break
print(j_number)

7


In [74]:
soup.findAll("script")[0]

<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
            new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
            j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
            '//www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
            })(window, document, 'script', 'dataLayer', 'GTM-MZPV7N');</script>

In [33]:
#все ссылки для последней страницы
js=str(soup.findAll("script")[7]).replace("<script>window.__INITIAL_DATA__ = ","").replace("</script>","")
import json
links=[]
for i in range(len(json.loads(js)['itemsState']['items'])):
    links.append('https://www.domofond.ru/'+json.loads(js)['itemsState']['items'][i]['itemUrl'])
links

['https://www.domofond.ru//2-komnatnaya-kvartira-na-prodazhu-moskva-2999223934',
 'https://www.domofond.ru//2-komnatnaya-kvartira-na-prodazhu-moskva-2815473102',
 'https://www.domofond.ru//2-komnatnaya-kvartira-na-prodazhu-moskva-2593843865',
 'https://www.domofond.ru//2-komnatnaya-kvartira-na-prodazhu-moskva-2593550226',
 'https://www.domofond.ru//4-komnatnaya-kvartira-na-prodazhu-moskva-2637189725',
 'https://www.domofond.ru//2-komnatnaya-kvartira-na-prodazhu-moskva-2800857599',
 'https://www.domofond.ru//2-komnatnaya-kvartira-na-prodazhu-moskva-2946192852',
 'https://www.domofond.ru//4-komnatnaya-kvartira-na-prodazhu-moskva-2989122773',
 'https://www.domofond.ru//2-komnatnaya-kvartira-na-prodazhu-moskva-2811679977',
 'https://www.domofond.ru//2-komnatnaya-kvartira-na-prodazhu-moskva-2812888393',
 'https://www.domofond.ru//2-komnatnaya-kvartira-na-prodazhu-moskva-2793356616',
 'https://www.domofond.ru//1-komnatnaya-kvartira-na-prodazhu-moskva-2955735613',
 'https://www.domofond.ru//1

In [7]:
#КОД ДЛЯ ЗАГРУЗКИ ВСЕХ ССЫЛОК
import json
import time
from bs4 import BeautifulSoup
links=[]
number=0
for i in range(1,2346):
    time.sleep(4)
    page_link='https://www.domofond.ru/prodazha-kvartiry-moskva-c3584?Rooms=One%2CTwo%2CThree%2CFour%2CFive%2CSix%2CSeven%2CEight%2CNine%2CMoreThan9%2CStudio&Page={}'.format(i)
    try:
        response=requests.get(page_link,headers={'User-Agent': UserAgent().chrome})
    except:
        print("Потеряно соединение с интернетом")
        time.sleep(60)
        response=requests.get(page_link,headers={'User-Agent': UserAgent().chrome})
    html=response.content
    soup = BeautifulSoup(html,"html.parser")
    j_number=0
    for j in range(len(soup.findAll("script"))):
        if str(soup.findAll("script")[j]).find("window.__INITIAL_DATA__ =")>0:
            j_number=j
            break
    try:
        js=str(soup.findAll("script")[j_number]).replace("<script>window.__INITIAL_DATA__ = ","").replace("</script>","")
        for i in range(len(json.loads(js)['itemsState']['items'])):
            links.append('https://www.domofond.ru/'+json.loads(js)['itemsState']['items'][i]['itemUrl'])
    except:
        print("ошибка")
        continue
    if number%25==0:
        print("скачано страниц:",number)
    number+=1

скачано страниц: 0
скачано страниц: 25
скачано страниц: 50
скачано страниц: 75
скачано страниц: 100
скачано страниц: 125
скачано страниц: 150
ошибка
скачано страниц: 175
скачано страниц: 200
скачано страниц: 225
скачано страниц: 250
скачано страниц: 275
скачано страниц: 300
скачано страниц: 325
ошибка
скачано страниц: 350
скачано страниц: 375
скачано страниц: 400
скачано страниц: 425
Потеряно соединение с интернетом
скачано страниц: 450
скачано страниц: 475
ошибка
скачано страниц: 500
скачано страниц: 525
ошибка
скачано страниц: 550
скачано страниц: 575
скачано страниц: 600
скачано страниц: 625
ошибка
скачано страниц: 650
ошибка
ошибка
скачано страниц: 675
скачано страниц: 700
скачано страниц: 725
скачано страниц: 750
скачано страниц: 775
скачано страниц: 800
ошибка
скачано страниц: 825
ошибка
скачано страниц: 850
ошибка
скачано страниц: 875
скачано страниц: 900
ошибка
ошибка
ошибка
скачано страниц: 925
скачано страниц: 950
скачано страниц: 975
Потеряно соединение с интернетом
скачано 

In [8]:
len(links)

62424

In [11]:
#import pandas as pd
#all_links=pd.DataFrame(links)
#all_links.to_excel("все ссылки НОВЫЕ.xlsx")

In [14]:
#all_links.drop_duplicates()

,0
0,https://www.domofond.ru//3-komnatnaya-kvartira...
1,https://www.domofond.ru//3-komnatnaya-kvartira...
2,https://www.domofond.ru//3-komnatnaya-kvartira...
3,https://www.domofond.ru//3-komnatnaya-kvartira...
4,https://www.domofond.ru//1-komnatnaya-kvartira...
...,...
62419,https://www.domofond.ru//1-komnatnaya-kvartira...
62420,https://www.domofond.ru//kvartira-na-prodazhu-...
62421,https://www.domofond.ru//2-komnatnaya-kvartira...
62422,https://www.domofond.ru//kvartira-na-prodazhu-...


In [15]:
last_page='https://www.domofond.ru/prodazha-kvartiry-moskva-c3584?Rooms=One%2CTwo%2CThree%2CFour%2CFive%2CSix%2CSeven%2CEight%2CNine%2CMoreThan9%2CStudio&Page=8'
response=requests.get(last_page,headers={'User-Agent': UserAgent().chrome})
html=response.content
soup = BeautifulSoup(html,"html.parser")
js=str(soup.findAll("script")[7]).replace("<script>window.__INITIAL_DATA__ = ","").replace("</script>","")
import json
links=[]
for i in range(len(json.loads(js)['itemsState']['items'])):
    links.append('https://www.domofond.ru/'+json.loads(js)['itemsState']['items'][i]['itemUrl'])
links

['https://www.domofond.ru//1-komnatnaya-kvartira-mfk_nahimov-moskva-u1001670164',
 'https://www.domofond.ru//5-komnatnaya-kvartira-na-prodazhu-moskva-2720307618',
 'https://www.domofond.ru//4-komnatnaya-kvartira-na-prodazhu-moskva-3023818384',
 'https://www.domofond.ru//2-komnatnaya-kvartira-na-prodazhu-moskva-2993248617',
 'https://www.domofond.ru//3-komnatnaya-kvartira-na-prodazhu-moskva-3023228801',
 'https://www.domofond.ru//1-komnatnaya-kvartira-mfk_nahimov-moskva-u1001670257',
 'https://www.domofond.ru//1-komnatnaya-kvartira-na-prodazhu-moskva-3022941784',
 'https://www.domofond.ru//3-komnatnaya-kvartira-na-prodazhu-moskva-2581872855',
 'https://www.domofond.ru//4-komnatnaya-kvartira-na-prodazhu-moskva-2945661843',
 'https://www.domofond.ru//1-komnatnaya-kvartira-na-prodazhu-moskva-2931630118',
 'https://www.domofond.ru//2-komnatnaya-kvartira-mfk_nahimov-moskva-u1001670417',
 'https://www.domofond.ru//2-komnatnaya-kvartira-na-prodazhu-moskva-3020578804',
 'https://www.domofond.ru

In [16]:
json.loads(js)['itemsState']['items'][0]['itemUrl']

'/1-komnatnaya-kvartira-mfk_nahimov-moskva-u1001670164'

In [17]:
#получаем часть html-файла с ссылками на квартиры, ища их по тегу a
hrefs=soup.findAll('a', attrs={'class':'long-item-card__item___ubItG search-results__itemCardNotFirst___3fei6'})

In [18]:
#список ссылок, которые получены из тега href и слеплены со второй частью ссылки
links=[]
for a in soup.find_all('a', href=True):
    if a['href'].find('kvartira-')>0:
        links.append('https://www.domofond.ru/'+a['href'])
links

['https://www.domofond.ru//1-komnatnaya-kvartira-mfk_nahimov-moskva-u1001670164',
 'https://www.domofond.ru//5-komnatnaya-kvartira-na-prodazhu-moskva-2720307618',
 'https://www.domofond.ru//4-komnatnaya-kvartira-na-prodazhu-moskva-3023818384',
 'https://www.domofond.ru//2-komnatnaya-kvartira-na-prodazhu-moskva-2993248617',
 'https://www.domofond.ru//3-komnatnaya-kvartira-na-prodazhu-moskva-3023228801',
 'https://www.domofond.ru//1-komnatnaya-kvartira-mfk_nahimov-moskva-u1001670257',
 'https://www.domofond.ru//1-komnatnaya-kvartira-na-prodazhu-moskva-3022941784',
 'https://www.domofond.ru//3-komnatnaya-kvartira-na-prodazhu-moskva-2581872855',
 'https://www.domofond.ru//4-komnatnaya-kvartira-na-prodazhu-moskva-2945661843',
 'https://www.domofond.ru//1-komnatnaya-kvartira-na-prodazhu-moskva-2931630118',
 'https://www.domofond.ru//2-komnatnaya-kvartira-mfk_nahimov-moskva-u1001670417',
 'https://www.domofond.ru//2-komnatnaya-kvartira-na-prodazhu-moskva-3020578804',
 'https://www.domofond.ru

In [19]:
#количество ссылок
len(links)

27

In [20]:
#собираем все данные по одной квартире
response=requests.get(links[0],headers={'User-Agent': UserAgent().chrome})
response

<Response [200]>

In [21]:
#для удобной работы с html обернем файл в Beautiful soup
soup=BeautifulSoup(response.content,"html.parser")

In [22]:
js=str(soup.findAll("script")[7]).replace("<script>window.__INITIAL_DATA__ = ","").replace("</script>","")
import json
json.loads(js)

{'itemsState': {'items': None,
  'currentItem': None,
  'currentItemError': None,
  'similarItems': None,
  'similarItemsSearchUrl': None,
  'nextPageItems': None,
  'breadCrumbs': None,
  'propertyTypeAnchors': None,
  'popularAreas': None,
  'surroundingAreas': None,
  'itemsForMap': None,
  'totalCount': None,
  'offset': None,
  'nextOffset': None,
  'nextLimit': None,
  'prevOffset': None,
  'pageSize': None,
  'searchUrl': None,
  'page': None,
  'prevSearchUrl': None,
  'nextSearchUrl': None,
  'pageHeading': None,
  'newItems': None,
  'shouldMapCorrectBounds': False,
  'searchAnchors': {}},
 'itemState': {'breadCrumbs': [{'title': 'Домофонд',
    'tooltip': 'Главная',
    'url': '/'},
   {'title': 'Москва',
    'tooltip': 'Купить квартиру в городе Москва, продажа квартир',
    'url': '/prodazha-kvartiry-moskva-c3584'},
   {'title': 'ЮЗАО',
    'tooltip': 'Купить квартиру в ЮЗАО, продажа квартир',
    'url': '/prodazha-kvartiry-yuzao-ad9'},
   {'title': 'Черёмушки',
    'toolti

Рассмотрим эти параметры, чтобы по ним построить модель:
<ol>
<li>цена (целевой фактор)</li>
<li>число комнат</li>
<li>является студией?</li>
<li>общая площадь</li>
<li>застройщик (если новостройка)</li>
<li>жилой комплекс</li>
<li>номер этажа квартиры</li>
<li>всего этажей</li>
<li>расстояние до центра города (через широту и долготу)</li>
<li>расстояние до ж/д станции в км и её название</li>
<li>расстояние до станции метро в км и её название</li>
<li>год постройки здания</li>
<li>вид материалов, из которых построено здание</li>
<li>первичный/вторичный рынок</li>
<li>есть телефон или нет</li>
<li>район города</li>
<li>описание</li>
<li>жилая площадь</li>
<li>площадь кухни</li>
<li>входит ли в программу реновации</li>
<li>Является ли предложением от агенства по продаже недвижимости</li>
</ol>

<i>(дополнительно)</i>

Возможно, стоит рассмотреть, продается ли квартира от агенства или нет, и название этого агенства

In [23]:
#цена
json.loads(js)['itemState']['item']['data']['priceValue']

10415710

In [24]:
#количество комнат
int(json.loads(js)['itemState']['item']['data']['rooms'])

KeyError: 'rooms'

In [25]:
#является студией?
json.loads(js)['itemState']['item']['data']['isStudio']

False

In [26]:
json.loads(js)['itemState']['item']['data'].keys()

dict_keys(['resultType', 'title', 'address', 'price', 'pricePerArea', 'priceValue', 'pricePerAreaValue', 'location', 'description', 'itemType', 'propertyType', 'isPrivate', 'isPremium', 'itemUrl', 'details', 'sourceKey', 'createdAt', 'updatedAt', 'sourceStartTime', 'sourceUpdateTime', 'region', 'city', 'adminDistrict', 'district', 'metro', 'developmentId', 'developmentName', 'closestStations', 'roomsOrdinal', 'isStudio', 'isFreePlan', 'floorAreaCalculated', 'floorInt', 'areaRating', 'areaFeedback', 'developerName', 'developmentHasContact', 'developmentContactName', 'developmentContactNumber', 'developmentContactHidePhone', 'developmentContactHide', 'developmentContactEmail', 'developmentProjectParticipant', 'developmentUrl', 'developmentDocuments'])

In [27]:
#общая площадь
json.loads(js)['itemState']['item']['data']['floorAreaCalculated']

50.1

In [28]:
#высота потолков - ТАКОГО ПАРАМЕТРА НЕТ

In [29]:
#застройщик (девелопер) - ТОЛЬКО ДЛЯ ПЕРВИЧНОГО РЫНКА
json.loads(js)['itemState']['item']['data']['developerName']

'ООО «ФЛЭТ и Ко» '

In [30]:
#название ЖК - в основном для первичного
json.loads(js)['itemState']['item']['data']['developmentName']

'МФК «Нахимов»'

In [31]:
#номер этажа квартиры
int(json.loads(js)['itemState']['item']['data']['floorString'].split('/')[0])

KeyError: 'floorString'

In [32]:
#всего этажей
int(json.loads(js)['itemState']['item']['data']['floorString'].split('/')[1])

KeyError: 'floorString'

In [33]:
#количество пассажирских лифтов - ЭТОЙ ИНФОРМАЦИИ ЧАСТО НЕТ

In [34]:
#координаты (долгода)
json.loads(js)['itemState']['item']['data']['location']['longitude']

37.58182

In [35]:
#координаты (широта)
json.loads(js)['itemState']['item']['data']['location']['latitude']

55.67059

In [36]:
#железнодорожные станции/станции метро (оставлю пока так, т.к. непонятно, что делать, если их несколько)
json.loads(js)['itemState']['item']['data']['closestStations']

[{'id': 112,
  'stationType': 'Metro',
  'name': 'Профсоюзная',
  'distance': 1.444,
  'distanceString': '1.4 км',
  'lineColors': ['#F58631']},
 {'id': 88,
  'stationType': 'Metro',
  'name': 'Новые Черемушки',
  'distance': 1.72,
  'distanceString': '1.7 км',
  'lineColors': ['#F58631']},
 {'id': 84,
  'stationType': 'Metro',
  'name': 'Нахимовский проспект',
  'distance': 1.736,
  'distanceString': '1.7 км',
  'lineColors': ['#A1A2A3']},
 {'id': 83,
  'stationType': 'Metro',
  'name': 'Нагорная',
  'distance': 1.837,
  'distanceString': '1.8 км',
  'lineColors': ['#A1A2A3']},
 {'id': 17,
  'stationType': 'Metro',
  'name': 'Академическая',
  'distance': 1.974,
  'distanceString': '2.0 км',
  'lineColors': ['#F58631']},
 {'id': 53,
  'stationType': 'Metro',
  'name': 'Каховская',
  'distance': 2.197,
  'distanceString': '2.2 км',
  'lineColors': ['#79CDCD']},
 {'id': 120,
  'stationType': 'Metro',
  'name': 'Севастопольская',
  'distance': 2.362,
  'distanceString': '2.4 км',
  'line

In [37]:
#парковка - ТАКОГО ЗНАЧЕНИЯ ЧАСТО НЕТ

In [38]:
#год постройки здания (если здание построено)
for i in range(len(json.loads(js)['itemState']['item']['data']['details'])):
    if str(json.loads(js)['itemState']['item']['data']['details'][i]).find('Год постройки')>0:
        print(json.loads(js)['itemState']['item']['data']['details'][i]['value'])

In [39]:
#у новостроек не указаны дедлайны по строительству, поэтому, возможно, стоит им присваивать значение последнего года?

In [40]:
#вид материалов, из которых построено здание (кирпичное, жд, панельное...)
for i in range(len(json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'])):
    if str(json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'][i]).find('Материал здания')>0:
        print(json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'][i]['value'])

KeyError: 'detailGroups'

In [41]:
#первичный или вторичный рынок
for i in range(len(json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'])):
    if str(json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'][i]).find('Тип объекта')>0:
        print(json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'][i]['value'])

KeyError: 'detailGroups'

In [42]:
#есть ли телефон?
json.loads(js)['itemState']['item']['phone']['exists']

True

In [43]:
#район города
json.loads(js)['itemState']['item']['data']['district']['name']

'Черёмушки'

In [44]:
#балкон - ПРО БАЛКОНЫ ИНФЫ НЕТ

In [45]:
#лоджия - ПРО ЛОДЖИИ ИНФЫ НЕТ

In [46]:
#описание
json.loads(js)['itemState']['item']['data']['description']

'<p>Многофункциональный комплекс «Нахимов» представляет собой три монолитных здания переменной этажности.</p><p>\xa0</p><p>Застройщиком спроектированы апартаменты с различными планировками: от одно- до трехкомнатных, общей площадью от 34,06 до 106,9 кв. м. Выполняется предчистовая отделка.</p><p>\xa0</p><p>Благоустройство прилегающей территории включает в себя организацию детских игровых и спортивных площадок, зон отдыха и озеленение. Предусмотрены открытые гостевые парковки и подземный паркинг. В шаговой доступности находятся детские сады, школы, медицинские учреждения, аптеки, продовольственные магазины, торгово-развлекательный центр и парки.</p><p>\xa0</p><p>Ввод объекта в эксплуатацию запланирован на 1 квартал 2021 года.</p>'

In [47]:
#вид из окна - НЕТ ИНФЫ

In [48]:
#вид ремонта - НЕТ ИНФЫ

In [49]:
#жилая площадь
for i in range(len(json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'])):
    if str(json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'][i]).find('Жилая площадь')>0:
        print(json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'][i]['value'])

KeyError: 'detailGroups'

In [50]:
#площадь кухни
for i in range(len(json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'])):
    if str(json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'][i]).find('Площадь кухни')>0:
        print(json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'][i]['value'])

KeyError: 'detailGroups'

In [51]:
#входит ли в программу реновации
json.loads(js)['itemState']['item']['data']['isPartOfRenovationProgram']

KeyError: 'isPartOfRenovationProgram'

In [52]:
#является ли предложением от агенства по продаже недвижимости
json.loads(js)['itemState']['item']['data']['agencyName']

KeyError: 'agencyName'

In [53]:
railways=[]
metro=[]
for i in range(len(json.loads(js)['itemState']['item']['data']['closestStations'])):
    if json.loads(js)['itemState']['item']['data']['closestStations'][i]['stationType']=='Railway':
        railways.append(json.loads(js)['itemState']['item']['data']['closestStations'][i])
    elif json.loads(js)['itemState']['item']['data']['closestStations'][i]['stationType']=='Metro':
        metro.append(json.loads(js)['itemState']['item']['data']['closestStations'][i])

In [54]:
metro

[{'id': 112,
  'stationType': 'Metro',
  'name': 'Профсоюзная',
  'distance': 1.444,
  'distanceString': '1.4 км',
  'lineColors': ['#F58631']},
 {'id': 88,
  'stationType': 'Metro',
  'name': 'Новые Черемушки',
  'distance': 1.72,
  'distanceString': '1.7 км',
  'lineColors': ['#F58631']},
 {'id': 84,
  'stationType': 'Metro',
  'name': 'Нахимовский проспект',
  'distance': 1.736,
  'distanceString': '1.7 км',
  'lineColors': ['#A1A2A3']},
 {'id': 83,
  'stationType': 'Metro',
  'name': 'Нагорная',
  'distance': 1.837,
  'distanceString': '1.8 км',
  'lineColors': ['#A1A2A3']},
 {'id': 17,
  'stationType': 'Metro',
  'name': 'Академическая',
  'distance': 1.974,
  'distanceString': '2.0 км',
  'lineColors': ['#F58631']},
 {'id': 53,
  'stationType': 'Metro',
  'name': 'Каховская',
  'distance': 2.197,
  'distanceString': '2.2 км',
  'lineColors': ['#79CDCD']},
 {'id': 120,
  'stationType': 'Metro',
  'name': 'Севастопольская',
  'distance': 2.362,
  'distanceString': '2.4 км',
  'line

In [55]:
railways

[{'id': 821,
  'stationType': 'Railway',
  'name': 'Нижние Котлы',
  'distance': 2.779,
  'distanceString': '2.8 км',
  'lineColors': ['#000000']}]

In [56]:
#создание однострочного датафрейма с информацией о ж/д станциях (т.к. их может быть несколько)
import numpy as np
from collections import defaultdict
super_dict = defaultdict(list)
for d in railways:
    for k, v in d.items():  # use d.iteritems() in python 2
        super_dict[k].append(v)
import pandas as pd
df_ravel=pd.DataFrame(pd.DataFrame(super_dict)[['name','distance']].values.ravel()).T
names=[]
for i in range(len(railways)):
    names.extend(["name railway station_{}".format(i), "distance railway station_{}".format(i)])
df_ravel.columns=names
df_ravel

,name railway station_0,distance railway station_0
0,Нижние Котлы,2.779


In [57]:
#то же самое со станциями метро
import numpy as np
from collections import defaultdict
super_dict = defaultdict(list)
for d in metro:
    for k, v in d.items():  # use d.iteritems() in python 2
        super_dict[k].append(v)
import pandas as pd
df_ravel=pd.DataFrame(pd.DataFrame(super_dict)[['name','distance']].values.ravel()).T
names=[]
for i in range(len(metro)):
    names.extend(["name metro station_{}".format(i), "distance metro station_{}".format(i)])
df_ravel.columns=names
df_ravel

,name metro station_0,distance metro station_0,name metro station_1,distance metro station_1,name metro station_2,distance metro station_2,name metro station_3,distance metro station_3,name metro station_4,distance metro station_4,name metro station_5,distance metro station_5,name metro station_6,distance metro station_6,name metro station_7,distance metro station_7,name metro station_8,distance metro station_8
0,Профсоюзная,1.444,Новые Черемушки,1.72,Нахимовский проспект,1.736,Нагорная,1.837,Академическая,1.974,Каховская,2.197,Севастопольская,2.362,Крымская,2.612,Нагатинская,2.976


In [58]:
import numpy as np
from collections import defaultdict
import pandas as pd

#обработчик ошибок для типа int
def except_int(x):
    try:
        return int(x)
    except:
        return np.nan

#теперь создадим один датафрейм со всеми факторами для одной квартиры
df=pd.DataFrame()
try:
    df['price']=[json.loads(js)['itemState']['item']['data']['priceValue']]
except:
    df['price']=[np.nan]
try:
    df['roomsCount']=[except_int(json.loads(js)['itemState']['item']['data']['rooms'])]
except:
    df['roomsCount']=[np.nan]
try:
    df['isStudio']=[json.loads(js)['itemState']['item']['data']['isStudio']]
except:
    df['isStudio']=[np.nan]
try:
    df['totalArea']=[json.loads(js)['itemState']['item']['data']['floorAreaCalculated']]
except:
    df['totalArea']=[np.nan]
try:
    df['developerName']=[json.loads(js)['itemState']['item']['data']['developerName']]
except:
    df['developerName']=[np.nan]
try:
    df['jkName']=[json.loads(js)['itemState']['item']['data']['developmentName']]
except:
    df['jkName']=[np.nan]
try:
    df['floorNumber']=[except_int(json.loads(js)['itemState']['item']['data']['floorString'].split('/')[0])]
except:
    df['floorNumber']=[np.nan]
try:
    df['floorsCount']=[except_int(json.loads(js)['itemState']['item']['data']['floorString'].split('/')[1])]
except:
    df['floorsCount']=[np.nan]
try:
    df['longitude']=[json.loads(js)['itemState']['item']['data']['location']['longitude']]
except:
    df['longitude']=[np.nan]
try:
    df['latitude']=[json.loads(js)['itemState']['item']['data']['location']['latitude']]
except:
    df['latitude']=[np.nan]

#делаем отдельный датафрейм для ж/д станций 

railways=[]
metro=[]
try:
    for i in range(len(json.loads(js)['itemState']['item']['data']['closestStations'])):
        try:
            if json.loads(js)['itemState']['item']['data']['closestStations'][i]['stationType']=='Railway':
                railways.append(json.loads(js)['itemState']['item']['data']['closestStations'][i])
        except:
            None
        try:
            if json.loads(js)['itemState']['item']['data']['closestStations'][i]['stationType']=='Metro':
                metro.append(json.loads(js)['itemState']['item']['data']['closestStations'][i])
        except:
            None
except:
    None

try:
    super_dict = defaultdict(list)
    for d in railways:
        for k, v in d.items():  # use d.iteritems() in python 2
            super_dict[k].append(v)
    df_ravel=pd.DataFrame(pd.DataFrame(super_dict)[['name','distance']].values.ravel()).T
    names=[]
    for i in range(len(railways)):
        names.extend(["name railway station_{}".format(i), "distance railway station_{}".format(i)])
    df_ravel.columns=names
    df=df.join(df_ravel)
except:
    None

#то же самое для станций метро

try:
    super_dict = defaultdict(list)
    for d in metro:
        for k, v in d.items():  # use d.iteritems() in python 2
            super_dict[k].append(v)
    df_ravel=pd.DataFrame(pd.DataFrame(super_dict)[['name','distance']].values.ravel()).T
    names=[]
    for i in range(len(metro)):
        names.extend(["name metro station_{}".format(i), "distance metro station_{}".format(i)])
    df_ravel.columns=names
    df=df.join(df_ravel)
except:
    None

try:
    for i in range(len(json.loads(js)['itemState']['item']['data']['details'])):
        if str(json.loads(js)['itemState']['item']['data']['details'][i]).find('Год постройки')>0:
            df['buildYear']=[json.loads(js)['itemState']['item']['data']['details'][i]['value']]
except:
    df['buildYear']=[np.nan]
try:
    for i in range(len(json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'])):
        if str(json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'][i]).find('Материал здания')>0:
            df['materialType']=[json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'][i]['value']]
except:
    df['materialType']=[np.nan]
try:
    for i in range(len(json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'])):
        if str(json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'][i]).find('Тип объекта')>0:
            df['offerType']=[json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'][i]['value']]
except:
    df['offerType']=[np.nan]
try:
    df['isPhone']=[json.loads(js)['itemState']['item']['phone']['exists']]
except:
    df['isPhone']=[np.nan]
try:
    df['rayon']=[json.loads(js)['itemState']['item']['data']['district']['name']]
except:
    df['rayon']=[np.nan]
try:
    df['description']=[json.loads(js)['itemState']['item']['data']['description']]
except:
    df['description']=[np.nan]
try:
    for i in range(len(json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'])):
        if str(json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'][i]).find('Жилая площадь')>0:
            df['livingArea']=[json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'][i]['value']]
except:
    df['livingArea']=[np.nan]
try:
    for i in range(len(json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'])):
        if str(json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'][i]).find('Площадь кухни')>0:
            df['kitchenArea']=[json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'][i]['value']]
except:
    df['kitchenArea']=[np.nan]
try:
    df['isPartOfRenovationProgram']=[json.loads(js)['itemState']['item']['data']['isPartOfRenovationProgram']]
except:
    df['isPartOfRenovationProgram']=[np.nan]
try:
    df['agencyName']=[json.loads(js)['itemState']['item']['data']['agencyName']]
except:
    df['agencyName']=[np.nan]

In [59]:
#делаем так, чтобы отображались все столбцы
pd.options.display.max_columns = 200
df

,price,roomsCount,isStudio,totalArea,developerName,jkName,floorNumber,floorsCount,longitude,latitude,name railway station_0,distance railway station_0,name metro station_0,distance metro station_0,name metro station_1,distance metro station_1,name metro station_2,distance metro station_2,name metro station_3,distance metro station_3,name metro station_4,distance metro station_4,name metro station_5,distance metro station_5,name metro station_6,distance metro station_6,name metro station_7,distance metro station_7,name metro station_8,distance metro station_8,materialType,offerType,isPhone,rayon,description,livingArea,kitchenArea,isPartOfRenovationProgram,agencyName
0,10415710,NaN,False,50.1,ООО «ФЛЭТ и Ко»,МФК «Нахимов»,NaN,NaN,37.58182,55.67059,Нижние Котлы,2.779,Профсоюзная,1.444,Новые Черемушки,1.72,Нахимовский проспект,1.736,Нагорная,1.837,Академическая,1.974,Каховская,2.197,Севастопольская,2.362,Крымская,2.612,Нагатинская,2.976,NaN,NaN,True,Черёмушки,<p>Многофункциональный комплекс «Нахимов» пред...,NaN,NaN,NaN,NaN


In [60]:
df.shape

(1, 39)

In [53]:
#скачиваем полученные ранее все ссылки на квартиры
#links=pd.read_excel("all_links.xlsx", index_col=0)

In [87]:
links=pd.read_excel('все ссылки НОВЫЕ.xlsx',index_col=0)

In [88]:
links=links.drop_duplicates()

In [89]:
#перемешиваем ссылки
links=list(pd.Series(links.iloc[:,0]))
from sklearn.utils import shuffle
links = shuffle(links)

In [90]:
len(links)

61647

In [91]:
links

['https://www.domofond.ru//2-komnatnaya-kvartira-na-prodazhu-moskva-3020996016',
 'https://www.domofond.ru//1-komnatnaya-kvartira-na-prodazhu-moskva-1223928925',
 'https://www.domofond.ru//1-komnatnaya-kvartira-na-prodazhu-moskva-2933632688',
 'https://www.domofond.ru//kvartira-na-prodazhu-moskva-3020693306',
 'https://www.domofond.ru//kvartira-na-prodazhu-moskva-3026946156',
 'https://www.domofond.ru//kvartira-na-prodazhu-moskva-2967016737',
 'https://www.domofond.ru//1-komnatnaya-kvartira-na-prodazhu-moskva-2531666768',
 'https://www.domofond.ru//kvartira-na-prodazhu-moskva-1677559006',
 'https://www.domofond.ru//1-komnatnaya-kvartira-na-prodazhu-moskva-3017395748',
 'https://www.domofond.ru//2-komnatnaya-kvartira-na-prodazhu-moskva-3034730473',
 'https://www.domofond.ru//2-komnatnaya-kvartira-na-prodazhu-moskva-1268910747',
 'https://www.domofond.ru//5-komnatnaya-kvartira-na-prodazhu-moskva-3034385411',
 'https://www.domofond.ru//2-komnatnaya-kvartira-na-prodazhu-moskva-2804992945',

In [92]:
#берем следующие значения с 1318 до 21600
links=links[1318:21600]
len(links)

20282

In [93]:
import numpy as np
from collections import defaultdict
import pandas as pd
import time

#обработчик ошибок для типа int
def except_int(x):
    try:
        return int(x)
    except:
        return np.nan
number=0
    
#делаем датафрейм для первой страницы
data=pd.DataFrame()
for href in links:
    if number%250==0:
        print("проверено {} квартир".format(number))
    #на случай, если пропадёт интернет, то подождать 2 минуты
    try:
        response=requests.get(href,headers={'User-Agent': UserAgent().chrome})
    except:
        print("Потеряно соединение с интернетом")
        time.sleep(60)
        response=requests.get(href,headers={'User-Agent': UserAgent().chrome})
    #если вдруг ссылка на квартиру была удалена или изменена, то мы её пропускаем
    if str(response)!='<Response [200]>':
        continue
    soup=BeautifulSoup(response.content,"html.parser")
    js=str(soup.findAll("script")[7]).replace("<script>window.__INITIAL_DATA__ = ","").replace("</script>","")
    import json

    #теперь создадим один датафрейм со всеми факторами для одной квартиры
    df=pd.DataFrame()
    try:
        df['price']=[json.loads(js)['itemState']['item']['data']['priceValue']]
    except:
        df['price']=[np.nan]
    try:
        df['roomsCount']=[except_int(json.loads(js)['itemState']['item']['data']['rooms'])]
    except:
        df['roomsCount']=[np.nan]
    try:
        df['isStudio']=[json.loads(js)['itemState']['item']['data']['isStudio']]
    except:
        df['isStudio']=[np.nan]
    try:
        df['totalArea']=[json.loads(js)['itemState']['item']['data']['floorAreaCalculated']]
    except:
        df['totalArea']=[np.nan]
    try:
        df['developerName']=[json.loads(js)['itemState']['item']['data']['developerName']]
    except:
        df['developerName']=[np.nan]
    try:
        df['jkName']=[json.loads(js)['itemState']['item']['data']['developmentName']]
    except:
        df['jkName']=[np.nan]
    try:
        df['floorNumber']=[except_int(json.loads(js)['itemState']['item']['data']['floorString'].split('/')[0])]
    except:
        df['floorNumber']=[np.nan]
    try:
        df['floorsCount']=[except_int(json.loads(js)['itemState']['item']['data']['floorString'].split('/')[1])]
    except:
        df['floorsCount']=[np.nan]
    try:
        df['longitude']=[json.loads(js)['itemState']['item']['data']['location']['longitude']]
    except:
        df['longitude']=[np.nan]
    try:
        df['latitude']=[json.loads(js)['itemState']['item']['data']['location']['latitude']]
    except:
        df['latitude']=[np.nan]

    #делаем отдельный датафрейм для ж/д станций 

    railways=[]
    metro=[]
    try:
        for i in range(len(json.loads(js)['itemState']['item']['data']['closestStations'])):
            try:
                if json.loads(js)['itemState']['item']['data']['closestStations'][i]['stationType']=='Railway':
                    railways.append(json.loads(js)['itemState']['item']['data']['closestStations'][i])
            except:
                None
            try:
                if json.loads(js)['itemState']['item']['data']['closestStations'][i]['stationType']=='Metro':
                    metro.append(json.loads(js)['itemState']['item']['data']['closestStations'][i])
            except:
                None
    except:
        None

    try:
        super_dict = defaultdict(list)
        for d in railways:
            for k, v in d.items():  # use d.iteritems() in python 2
                super_dict[k].append(v)
        df_ravel=pd.DataFrame(pd.DataFrame(super_dict)[['name','distance']].values.ravel()).T
        names=[]
        for i in range(len(railways)):
            names.extend(["name railway station_{}".format(i), "distance railway station_{}".format(i)])
        df_ravel.columns=names
        df=df.join(df_ravel)
    except:
        None

    #то же самое для станций метро

    try:
        super_dict = defaultdict(list)
        for d in metro:
            for k, v in d.items():  # use d.iteritems() in python 2
                super_dict[k].append(v)
        df_ravel=pd.DataFrame(pd.DataFrame(super_dict)[['name','distance']].values.ravel()).T
        names=[]
        for i in range(len(metro)):
            names.extend(["name metro station_{}".format(i), "distance metro station_{}".format(i)])
        df_ravel.columns=names
        df=df.join(df_ravel)
    except:
        None

    try:
        for i in range(len(json.loads(js)['itemState']['item']['data']['details'])):
            if str(json.loads(js)['itemState']['item']['data']['details'][i]).find('Год постройки')>0:
                df['buildYear']=[json.loads(js)['itemState']['item']['data']['details'][i]['value']]
    except:
        df['buildYear']=[np.nan]
    try:
        for i in range(len(json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'])):
            if str(json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'][i]).find('Материал здания')>0:
                df['materialType']=[json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'][i]['value']]
    except:
        df['materialType']=[np.nan]
    try:
        for i in range(len(json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'])):
            if str(json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'][i]).find('Тип объекта')>0:
                df['offerType']=[json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'][i]['value']]
    except:
        df['offerType']=[np.nan]
    try:
        df['isPhone']=[json.loads(js)['itemState']['item']['phone']['exists']]
    except:
        df['isPhone']=[np.nan]
    try:
        df['rayon']=[json.loads(js)['itemState']['item']['data']['district']['name']]
    except:
        df['rayon']=[np.nan]
    try:
        df['description']=[json.loads(js)['itemState']['item']['data']['description']]
    except:
        df['description']=[np.nan]
    try:
        for i in range(len(json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'])):
            if str(json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'][i]).find('Жилая площадь')>0:
                df['livingArea']=[json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'][i]['value']]
    except:
        df['livingArea']=[np.nan]
    try:
        for i in range(len(json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'])):
            if str(json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'][i]).find('Площадь кухни')>0:
                df['kitchenArea']=[json.loads(js)['itemState']['item']['data']['detailGroups'][0]['details'][i]['value']]
    except:
        df['kitchenArea']=[np.nan]
    try:
        df['isPartOfRenovationProgram']=[json.loads(js)['itemState']['item']['data']['isPartOfRenovationProgram']]
    except:
        df['isPartOfRenovationProgram']=[np.nan]
    
    #присоединяем данные, полученные по одной квартире, к датафрейму    
    data=pd.concat([data,df])
    number+=1
    
    time.sleep(4)

проверено 0 квартир
проверено 250 квартир
проверено 500 квартир
проверено 750 квартир
Потеряно соединение с интернетом
проверено 1000 квартир
Потеряно соединение с интернетом
проверено 1250 квартир
Потеряно соединение с интернетом
проверено 1500 квартир
проверено 1750 квартир
проверено 2000 квартир
проверено 2250 квартир
проверено 2500 квартир
проверено 2750 квартир
проверено 3000 квартир
проверено 3250 квартир
проверено 3500 квартир
проверено 3750 квартир
проверено 4000 квартир
проверено 4250 квартир
проверено 4500 квартир
проверено 4750 квартир
проверено 5000 квартир
проверено 5250 квартир
проверено 5500 квартир
проверено 5750 квартир
проверено 6000 квартир
проверено 6250 квартир
проверено 6500 квартир
проверено 6750 квартир
проверено 7000 квартир
проверено 7250 квартир
проверено 7500 квартир
проверено 7750 квартир
проверено 8000 квартир
Потеряно соединение с интернетом
проверено 8250 квартир
Потеряно соединение с интернетом
проверено 8500 квартир
проверено 8750 квартир
проверено 900

In [94]:
data

,price,roomsCount,isStudio,totalArea,developerName,jkName,floorNumber,floorsCount,longitude,latitude,name railway station_0,distance railway station_0,name metro station_0,distance metro station_0,name metro station_1,distance metro station_1,name metro station_2,distance metro station_2,name metro station_3,distance metro station_3,name metro station_4,distance metro station_4,name metro station_5,distance metro station_5,name metro station_6,distance metro station_6,name metro station_7,distance metro station_7,name metro station_8,distance metro station_8,buildYear,materialType,offerType,isPhone,rayon,description,livingArea,kitchenArea,isPartOfRenovationProgram,name railway station_1,distance railway station_1,name railway station_2,distance railway station_2,name railway station_3,distance railway station_3,name railway station_4,distance railway station_4,name metro station_9,distance metro station_9,name railway station_5,distance railway station_5,name railway station_6,distance railway station_6,name railway station_7,distance railway station_7
0,79530000.0,3.0,0.0,143.0,АО «ИНТЕКО»,ЖК «Садовые Кварталы»,2.0,14.0,37.570500,55.725610,Москва Киевская,1.888,Спортивная,0.609,Фрунзенская,0.643,Лужники,0.828,Парк культуры,1.763,Воробьевы Горы,1.83,Студенческая,2.021,Киевская,2.072,Ленинский проспект,2.227,Площадь Гагарина,2.281,NaN,Монолит,Новостройка,1.0,Хамовники,«Садовые кварталы» - уникальная концепция прем...,,,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,31675855.0,2.0,0.0,106.0,ООО «Бакунинская»,Апартаменты «TATLIN APARTMENTS» (Татлин Апартм...,8.0,12.0,37.679560,55.773980,Москва Казанская,1.233,Бауманская,0.178,Красносельская,1.069,Комсомольская,1.567,Сокольники,1.694,Электрозаводская,1.848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Монолит,Новостройка,1.0,Басманный,"Предлагается 2-комнатные апартаменты, общей пл...",,,0.0,Москва Ярославская,1.513,Москва Ленинградская,1.709,Каланчёвская,1.781,Электрозаводская,1.804,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,8700000.0,2.0,0.0,41.4,NaN,NaN,2.0,17.0,37.191187,55.965546,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014,Монолит,Вторичная,1.0,Зеленоград,Евродвушка на базе 1-комнатной квартиры свобод...,29 м²,8 м²,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,27500000.0,3.0,0.0,80.0,NaN,NaN,5.0,8.0,37.555609,55.747897,Москва Киевская,0.798,Киевская,0.803,Выставочная,0.812,Деловой центр,1.02,Студенческая,1.112,Международная,1.42,Деловой центр (МЦК),1.489,Кутузовская,1.594,Смоленская,1.641,NaN,NaN,1957,Кирпич,Вторичная,1.0,Дорогомилово,Лот 62247. В западном округе Москвы в кирпично...,,,0.0,Тестовская,1.658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,26500000.0,3.0,0.0,90.2,NaN,NaN,9.0,14.0,37.636845,55.699694,ЗИЛ,0.98,ЗИЛ,0.789,Тульская,1.356,Автозаводская,1.485,Верхние котлы,1.593,Технопарк,1.796,Нагатинская,1.983,Крымская,2.265,NaN,NaN,NaN,NaN,NaN,Монолит,Вторичная,1.0,Даниловский,"Id 18914. ВНИМАНИЕ, СУПЕР ПРЕДЛОЖЕНИЕ. Предлаг...",46.1 м²,20.5 м²,0.0,Москва Товарная,1.322,Нижние Котлы,2.186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,42500000.0,4.0,0.0,120.0,NaN,NaN,6.0,6.0,37.652589,55.757577,Москва Курская,0.603,Чкаловская,0.457,Курская,0.493,Чистые пруды,1.186,Китай-город,1.228,Красные ворота,1.27,Тургеневская,1.313,Сретенский бульвар,1.407,Лубянка,1.595,Кузнецкий мост,1.815,1914,Кирпич,Вторичная,1.0,Басманный,"Номер лота: 3485586, (Александр) Невероятно ст...",85 м²,9 м²,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,7468313.0,1.0,0.0,48.5,Компания «НДВ-Супермаркет Недвижимости»,Комплекс апартаментов «Nord» (Норд),11.0,11.0,37.541100,55.920530,Марк,1.793,Алтуфьево,3.801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Мо

In [95]:
data.drop_duplicates().shape

(19791, 55)

In [83]:
#data.to_excel("первые 1318 квартир.xlsx")

In [96]:
#data.to_excel("следующие 20275 квартир.xlsx")

<b>Необходимо избежать ошибок, связанных со ссылками - некоторые из них могут быть удалены, тогда возникнет ошибка. Будем сравнивать значение ответа сервера с '<Response [200]>' и в случае несовпадения продолжать цикл с новой ссылки при помощи continue</b>

В целом за 24 часа можно скачать 21600 ссылок

In [103]:
data_old=pd.read_excel("первые 1318 квартир.xlsx", index_col=0)

In [104]:
final_data=pd.concat([data,data_old])

In [106]:
final_data=final_data.drop_duplicates()
final_data.shape

(20965, 55)

In [107]:
#final_data.to_excel("финальные данные 2.xlsx")